In [14]:
import pandas as pd

In [15]:
df0 = pd.read_csv("superstore.csv")
display(df0.shape)
df0.head(2)

(10000, 20)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,ES-2017-1311038,07/02/2017,11/02/2017,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,OFF-ST-10000988,Office Supplies,Storage,"Fellowes Folders, Blue",79.20,3,0.0,39.6
1,2,ES-2017-1311038,07/02/2017,11/02/2017,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,TEC-AC-10004144,Technology,Accessories,"SanDisk Numeric Keypad, Bluetooth",388.92,7,0.0,0.0


In [16]:
# formatting column names
df0.columns = df0.columns.str.lower().str.replace("-","_").str.replace(" ","_")
df0.head(2)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,ES-2017-1311038,07/02/2017,11/02/2017,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,OFF-ST-10000988,Office Supplies,Storage,"Fellowes Folders, Blue",79.20,3,0.0,39.6
1,2,ES-2017-1311038,07/02/2017,11/02/2017,Standard Class,AS-10045,Aaron Smayling,Corporate,Leeds,England,United Kingdom,North,TEC-AC-10004144,Technology,Accessories,"SanDisk Numeric Keypad, Bluetooth",388.92,7,0.0,0.0


In [17]:
# 
df = df0[["order_date","product_id","sub_category","product_name","sales","quantity"]]
import warnings
# from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
# warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
df["order_date"] = pd.to_datetime(df["order_date"])
df.head(2)

C:\Users\mdima\AppData\Local\Temp\ipykernel_6788\184534323.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["order_date"] = pd.to_datetime(df["order_date"])


,order_date,product_id,sub_category,product_name,sales,quantity
0,2017-07-02,OFF-ST-10000988,Storage,"Fellowes Folders, Blue",79.20,3
1,2017-07-02,TEC-AC-10004144,Accessories,"SanDisk Numeric Keypad, Bluetooth",388.92,7


In [18]:
# creating dataframe with only unique product names
lname,lcat,lid,lbuy = [],[],[],[]
for i in range(df.shape[0]):
    if df.iloc[i]["product_name"] not in lname:
        lname.append(df.iloc[i]["product_name"])
        lid.append(df.iloc[i]["product_id"])
        lcat.append(df.iloc[i]["sub_category"])
        lbuy.append((df.iloc[i]["sales"]/df.iloc[i]["quantity"]).round(0)-0.01)
    
print(len(lname), len(lcat), len(lid), len(lbuy))

1856 1856 1856 1856


In [19]:
# dataframe: df logistics (dflog)
import random
dflog = pd.DataFrame()
dflog["product_name"] = lname
dflog["product_id"] = lid
dflog["category"] = lcat
dflog["buy_price"] = lbuy
lsell,lstock = [],[]
for i in dflog["buy_price"]:
    lsell.append(round((i + (i*(random.randint(10,30)/100))),2))
    lstock.append(random.randint(0,20))
dflog["sell_price"] = lsell
dflog["stock"] = lstock
dflog["profit_per_item"] = ((dflog["sell_price"] - dflog["buy_price"])).round(2)
dflog.head()

,product_name,product_id,category,buy_price,sell_price,stock,profit_per_item
0,"Fellowes Folders, Blue",OFF-ST-10000988,Storage,25.99,33.53,16,7.54
1,"SanDisk Numeric Keypad, Bluetooth",TEC-AC-10004144,Accessories,55.99,64.95,10,8.96
2,"Avery Legal Exhibit Labels, 5000 Label Set",OFF-LA-10001915,Labels,11.99,14.51,7,2.52
3,"Fellowes Folders, Wire Frame",OFF-ST-10004550,Storage,24.99,31.49,11,6.50
4,"Memorex Memory Card, USB",TEC-AC-10004068,Accessories,101.99,121.37,6,19.38


In [20]:
# reshaping/manipulating dataframe: store_data
df = pd.merge(df,dflog[["product_name","buy_price","sell_price","profit_per_item"]], on = "product_name", how = "left")
df = df.rename(columns = {"order_date":"date","sub_category":"category"})
df['date'] = df['date'] + pd.offsets.DateOffset(years=5)
df["date"] = df["date"] + pd.DateOffset(months = 4)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
# df['month'] = pd.to_datetime(df['month'], format='%m').dt.strftime('%B')
df['year'] = df['date'].dt.year

df["sales"] = df["buy_price"] * df["quantity"]
df["profit"] = (df["profit_per_item"] * df["quantity"]).round(2)

df = df[["date","month","year","product_id","product_name","category","buy_price","sell_price","quantity","sales","profit_per_item","profit"]]
df.head(2)

,date,month,year,product_id,product_name,category,buy_price,sell_price,quantity,sales,profit_per_item,profit
0,2022-11-02,11,2022,OFF-ST-10000988,"Fellowes Folders, Blue",Storage,25.99,33.53,3,77.97,7.54,22.62
1,2022-11-02,11,2022,TEC-AC-10004144,"SanDisk Numeric Keypad, Bluetooth",Accessories,55.99,64.95,7,391.93,8.96,62.72


In [21]:
dflog.head()

,product_name,product_id,category,buy_price,sell_price,stock,profit_per_item
0,"Fellowes Folders, Blue",OFF-ST-10000988,Storage,25.99,33.53,16,7.54
1,"SanDisk Numeric Keypad, Bluetooth",TEC-AC-10004144,Accessories,55.99,64.95,10,8.96
2,"Avery Legal Exhibit Labels, 5000 Label Set",OFF-LA-10001915,Labels,11.99,14.51,7,2.52
3,"Fellowes Folders, Wire Frame",OFF-ST-10004550,Storage,24.99,31.49,11,6.50
4,"Memorex Memory Card, USB",TEC-AC-10004068,Accessories,101.99,121.37,6,19.38


In [22]:
df.to_csv("store_data.csv", index = False)
dflog.to_csv("logistics.csv", index = False)

In [23]:
df.dtypes

date               datetime64[ns]
month                       int64
year                        int64
product_id                 object
product_name               object
category                   object
buy_price                 float64
sell_price                float64
quantity                    int64
sales                     float64
profit_per_item           float64
profit                    float64
dtype: object

In [24]:
df.date.max()

Timestamp('2023-04-30 00:00:00')